In [1]:
import os

import snowflake.connector

/opt/conda/lib/python3.7/site-packages/snowflake/connector/options.py:97: UserWarning: You have an incompatible version of 'pyarrow' installed (4.0.0), please install a version that adheres to: 'pyarrow<5.1.0,>=5.0.0; extra == "pandas"'
  _expected_pyarrow_version,


In [3]:
con = snowflake.connector.connect(
    user=os.environ.get("SNOWFLAKE_USER"),
    password=os.environ.get("SNOWFLAKE_PASSWORD"),
    account=os.environ.get("SNOWFLAKE_ACCOUNT"),
    warehouse=os.environ.get("SNOWFLAKE_WAREHOUSE"),
)

In [4]:
db_exists = False

try:
    con.cursor().execute("USE DATABASE diepvries_tutorial")
    db_exists = True
except Exception:
    print("Database doesn't exist yet. Creating...")

Database doesn't exist yet. Creating...


In [5]:
queries = [
    """CREATE DATABASE diepvries_tutorial;""",
    """USE DATABASE diepvries_tutorial;""",
    """CREATE SCHEMA dv;""",
    """CREATE SCHEMA dv_extract;""",
    """CREATE SCHEMA dv_staging;""",
    """CREATE TABLE IF NOT EXISTS dv.h_customer (
  h_customer_hashkey VARCHAR(32) NOT NULL UNIQUE COMMENT 'Record hashkey',
  r_timestamp        TIMESTAMP   NOT NULL COMMENT 'Record timestamp',
  r_source           VARCHAR     NOT NULL COMMENT 'Record source',
  customer_id        VARCHAR     NOT NULL COMMENT 'Customer business ID',
  PRIMARY KEY (h_customer_hashkey)
);""",
    """CREATE TABLE IF NOT EXISTS dv.hs_customer (
  h_customer_hashkey VARCHAR(32) NOT NULL REFERENCES dv.h_customer (h_customer_hashkey) COMMENT 'Record hashkey',
  s_hashdiff         VARCHAR(32) NOT NULL COMMENT 'Record hashdiff',
  r_timestamp        TIMESTAMP   NOT NULL COMMENT 'Record start timestamp',
  r_timestamp_end    TIMESTAMP   NOT NULL COMMENT 'Record end timestamp',
  r_source           VARCHAR     NOT NULL COMMENT 'Record source',
  firstname          VARCHAR COMMENT 'Customer first name',
  lastname           VARCHAR COMMENT 'Customer last name',
  PRIMARY KEY (h_customer_hashkey, r_timestamp)
);""",
    """CREATE TABLE IF NOT EXISTS dv.h_order (
  h_order_hashkey VARCHAR(32) NOT NULL UNIQUE COMMENT 'Record hashkey',
  r_timestamp     TIMESTAMP   NOT NULL COMMENT 'Record timestamp',
  r_source        VARCHAR     NOT NULL COMMENT 'Record source',
  order_id        VARCHAR     NOT NULL COMMENT 'Order business ID',
  PRIMARY KEY (h_order_hashkey)
);""",
    """CREATE TABLE IF NOT EXISTS dv.hs_order (
  h_order_hashkey VARCHAR(32) NOT NULL REFERENCES dv.h_order (h_order_hashkey) COMMENT 'Record hashkey',
  s_hashdiff      VARCHAR(32) NOT NULL COMMENT 'Record hashdiff',
  r_timestamp     TIMESTAMP   NOT NULL COMMENT 'Record start timestamp',
  r_timestamp_end TIMESTAMP   NOT NULL COMMENT 'Record end timestamp',
  r_source        VARCHAR     NOT NULL COMMENT 'Record source',
  create_ts       TIMESTAMP COMMENT 'Order creation timestamp',
  quantity        INTEGER COMMENT 'Order quantity',
  PRIMARY KEY (h_order_hashkey, r_timestamp)
);""",
    """CREATE TABLE IF NOT EXISTS dv.l_order_customer (
  l_order_customer_hashkey VARCHAR(32) NOT NULL COMMENT 'Record hashkey',
  h_customer_hashkey       VARCHAR(32) NOT NULL REFERENCES dv.h_customer (h_customer_hashkey) COMMENT 'Customer hashkey',
  h_order_hashkey          VARCHAR(32) NOT NULL REFERENCES dv.h_order (h_order_hashkey) COMMENT 'Order hashkey',
  customer_id              VARCHAR     NOT NULL COMMENT 'Customer business ID',
  order_id                 VARCHAR     NOT NULL COMMENT 'Order business ID',
  r_timestamp              TIMESTAMP   NOT NULL COMMENT 'Record timestamp',
  r_source                 VARCHAR     NOT NULL COMMENT 'Record source',
  PRIMARY KEY (l_order_customer_hashkey)
);""",
    """CREATE OR REPLACE TABLE dv_extract.order_customer (
  customer_id VARCHAR,
  firstname   VARCHAR,
  lastname    VARCHAR,
  order_id    VARCHAR,
  create_ts   VARCHAR,
  quantity    VARCHAR
);""",
    """INSERT INTO dv_extract.order_customer
VALUES
  ('1', 'Alice', 'Doe', '1', '2021-03-17T14:00:00+00:00', '2'),
  ('1', 'Alice', 'Doe', '2', '2021-03-17T15:00:00+00:00', '3'),
  ('1', 'Alice', 'Doe', '3', '2021-03-17T16:00:00+00:00', '2'),
  ('2', 'Bob', 'Smith', '4', '2021-03-17T17:00:00+00:00', '8'),
  (NULL, 'Charlie', NULL, '4', '2021-03-17T18:00:00+00:00', '8');
""",
]

In [6]:
if not db_exists:
    for query in queries:
        print(query)
        con.cursor().execute(query)

CREATE DATABASE diepvries_tutorial;
USE DATABASE diepvries_tutorial;
CREATE SCHEMA dv;
CREATE SCHEMA dv_extract;
CREATE SCHEMA dv_staging;
CREATE TABLE IF NOT EXISTS dv.h_customer (
  h_customer_hashkey VARCHAR(32) NOT NULL UNIQUE COMMENT 'Record hashkey',
  r_timestamp        TIMESTAMP   NOT NULL COMMENT 'Record timestamp',
  r_source           VARCHAR     NOT NULL COMMENT 'Record source',
  customer_id        VARCHAR     NOT NULL COMMENT 'Customer business ID',
  PRIMARY KEY (h_customer_hashkey)
);
CREATE TABLE IF NOT EXISTS dv.hs_customer (
  h_customer_hashkey VARCHAR(32) NOT NULL REFERENCES dv.h_customer (h_customer_hashkey) COMMENT 'Record hashkey',
  s_hashdiff         VARCHAR(32) NOT NULL COMMENT 'Record hashdiff',
  r_timestamp        TIMESTAMP   NOT NULL COMMENT 'Record start timestamp',
  r_timestamp_end    TIMESTAMP   NOT NULL COMMENT 'Record end timestamp',
  r_source           VARCHAR     NOT NULL COMMENT 'Record source',
  firstname          VARCHAR COMMENT 'Customer fir